In [1]:
from temp import *


In [2]:
class Generator(nn.Module):
    def __init__(self,z_dim,y_dim,h_dim):
        super(Generator,self).__init__()
        
        self.layer = nn.Sequential(
            nn.Linear(z_dim + y_dim, h_dim*4),
            Unflatten(C=h_dim, W=2, H=2),
            *block_t(h_dim,h_dim//2),
            *block_t(h_dim//2,h_dim//4),
            *block_t(h_dim//4,h_dim//8),
            *block_t(h_dim//8,1,nn.Sigmoid())
        )
        
        initialize_weights(self)
    
    def forward(self,z,y):
        return self.layer(T.cat([z,y],1))
    
class Encoder(nn.Module):
    def __init__(self,z_dim,y_dim,h_dim):
        super(Encoder,self).__init__()

        self.z_dim = z_dim
        
        self.layer = nn.Sequential(
            *block(1,h_dim//8),#32>16
            *block(h_dim//8,h_dim//4),#16>8
            *block(h_dim//4,h_dim//2),#8>4
            *block(h_dim//2,h_dim),#4>2
            Flatten(),
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(h_dim*4+y_dim, h_dim*2),
            nn.ReLU(),
            nn.Linear(h_dim*2,z_dim*2),
        )
        
        initialize_weights(self)
        
    def reparametrize(self,mu,scale):
        eps = T.randn_like(T.exp(scale))
        z = mu + eps * T.exp(scale/2)
        return z

    def forward(self,x,y):
        h = self.layer(x)
        h = self.fc_layer(T.cat([h,y],1))
        mu = h[:,:self.z_dim]
        scale = h[:,self.z_dim:]
        kl = - 0.5* T.sum(1 + scale - mu.pow(2) - scale.exp())/T.numel(mu)
        return self.reparametrize(mu,scale), kl

class Discriminator(nn.Module):
    def __init__(self,h_dim):
        super(Discriminator,self).__init__()

        self.layer = nn.Sequential(
            *block(1,h_dim//8,nn.LeakyReLU(0.2)),#32>16
            *block(h_dim//8,h_dim//4,nn.LeakyReLU(0.2)),#16>8
            *block(h_dim//4,h_dim//2,nn.LeakyReLU(0.2)),#8>4
            *block(h_dim//2,h_dim,nn.LeakyReLU(0.2)),#4>2
            Flatten(),
            nn.Linear(h_dim*4, 1),
            nn.Sigmoid()
        )
        
        initialize_weights(self)

    def forward(self,x):
        return self.layer(x)

class VAE_GAN(nn.Module):
    def __init__(self,E,G,D,GP):
        super(VAE_GAN,self).__init__()
        
        self.z_dim = E.z_dim
        self.E = E
        self.G = G
        self.D = D
        self.GP = GP
        
    def forward(self,x,y):
        z, kl = self.E(x,y)
        x_f = self.G(z,y)
        return x_f
        
    def generate_random(self,y):
        z = T.randn(y.shape[0],self.z_dim)
        x = self.G(z,y)
        return x
    
    def calc_gp(self,x, x_p):
        alpha = T.rand(x.shape[0], 1, 1, 1).expand_as(x)

        ip = (alpha * x.data + (1 - alpha) * x_p.data)
        ip.requires_grad = True

        prob_ip = self.D(ip)

        grad = T.autograd.grad(outputs=prob_ip,
                               inputs=ip,
                               grad_outputs=T.ones(
                                   prob_ip.size()),
                               create_graph=True,
                               retain_graph=True)[0].view(x.shape[0],-1)

        grad_norm = T.sqrt(T.sum(grad ** 2, dim=1) + 1e-12)
        return self.GP * ((grad_norm - 1) ** 2).mean()
    
    def d_loss(self,x,y):
        x_p = self.generate_random(y).detach()
        
        d_p = self.D(x_p).mean()
        d_r = self.D(x).mean()
        GP = self.calc_gp(x,x_p)
                
        return {'d_p':d_p,'d_r':d_r,'GP':GP}
    
    def g_loss(self,x,y):
        z, kl = self.E(x,y)
        x_f = self.G(z,y)
        x_p = self.generate_random(y)
        
        recon = (x - x_f).pow(2).mean()
        d_p = self.D(x_p).mean()
        
        return {'kl':kl,'recon':recon,'d_p':d_p}


In [3]:
def train_test(model,opt,loaders,writer,epoch,mode):
    loss = {i: T.tensor(0.) for i in losses}
    cnt = 0
    div = 0
    for loader in tqdm(loaders):
        for x,y in loader:
            cnt += 1
            x = x.to(device)
            y = y.to(device)
            
            ret = model.d_loss(x,y)
            d_loss = ret['d_p'] - ret['d_r'] + ret['GP']
            
            if mode == 'train':
                opt['D'].zero_grad()
                d_loss.backward()
                opt['D'].step()

            for i in ret.keys():
                loss[i] += ret[i]

            if cnt % n_disc == 0:
                div += 1
                ret = model.g_loss(x,y)
                g_loss = ret['kl'] + ret['recon'] - gamma*ret['d_p']
                if mode == 'train':
                    opt['E'].zero_grad()
                    opt['G'].zero_grad()
                    g_loss.backward()
                    opt['E'].step()                    
                    opt['G'].step()
    
                for i in ret.keys():
                    if i != 'd_p':
                        loss[i] += ret[i]
    
    for i in loss.keys():
        if i == 'kl' or i == 'recon':
            loss[i] /= div
        else:
            loss[i] /= cnt
    upload(writer,loss,epoch,mode)
    
    return loss

In [4]:
root = '/Users/taurine511/Desktop/abc32'
models = '/Users/taurine511/Desktop/models'

name = 'VAE+GlyphGAN'
batch_size = 128
epochs = 200

seed = 1
T.manual_seed(seed)

writer = Writer(name)
gamma = 1
y_dim = 26
z_dim = 100
h_dim = 128
n_disc = 5
GP_weight = 10
losses = ['kl','recon','d_p','d_r','GP']

if T.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [5]:
train_loaders, test_loaders = get_loaders(root,y_dim,batch_size)

In [6]:
E = Encoder(z_dim,y_dim,h_dim).to(device)
G = Generator(z_dim,y_dim,h_dim).to(device)
D = Discriminator(h_dim).to(device)
model = VAE_GAN(E,G,D,GP_weight)
opt = {}
opt['E'] = optim.Adam(E.parameters(),lr=2e-4,betas=(0.5,0.99))
opt['G'] = optim.Adam(G.parameters(),lr=2e-4,betas=(0.5,0.99))
opt['D'] = optim.Adam(D.parameters(),lr=2e-4,betas=(0.5,0.99))

In [7]:
label = T.zeros(64).to(T.int64).to(device)
label = F.one_hot(label, y_dim)
z_sample = T.randn(64,z_dim)
sample = G(z_sample,label)
x_fix,y_fix = next(iter(train_loaders[0]))

In [8]:
for epoch in range(1, epochs + 1):
    print(f'Epoch {epoch}')
    if epoch % 10 == 1:
        T.save(model.state_dict(),f'{models}/{name}.pth')
    
    train_test(model,opt,test_loaders,writer,epoch,'train')
    train_test(model,opt,test_loaders,writer,epoch,'test')
    
    sample = G(z_sample,label)
    recon = model(x_fix[:8],y_fix[:8])
    writer.add_images('Fixed_recon',T.cat([x_fix[:8],recon],2), epoch)
    writer.add_images('Image_from_latent', sample, epoch)
    
writer.close()




Epoch 1


100%|███████████████████████████████████████████| 26/26 [02:11<00:00,  5.06s/it]


Epoch 2


100%|███████████████████████████████████████████| 26/26 [02:20<00:00,  5.39s/it]


Epoch 3


...

100%|███████████████████████████████████████████| 26/26 [02:07<00:00,  4.91s/it]


Epoch 199


100%|███████████████████████████████████████████| 26/26 [02:00<00:00,  4.64s/it]


Epoch 200


100%|███████████████████████████████████████████| 26/26 [01:38<00:00,  3.77s/it]
